### stage_model_classifier

prediction models of sign classifiers on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# to use only one GPU.
# use this on r-001
# otherwise comment
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb


In [3]:
# bulid regression model
classifiers = ['currank','avgrank','dice','lr','lrl1','lsvc','lsvcl2','rf','lrbias','xgb']
def get_classifier(classifier = 'lr'):
    
    class_weight = None
    
    if classifier == "lsvc":
        clf = LinearSVC(penalty='l1',dual=False, tol=1e-3, class_weight=class_weight )
    elif classifier == "lsvcl2":
        clf = LinearSVC(penalty='l2', tol=1e-4, class_weight=class_weight)
    elif classifier == 'rf':
        #clf = RandomForestClassifier(n_estimators=100, n_jobs=4,criterion='entropy', min_samples_split=1,class_weight = class_weight)
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1,criterion='entropy', class_weight = class_weight)
    elif classifier == 'lr':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = False, verbose = 0)
    elif classifier == 'lrbias':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = True, verbose = 1)
    elif classifier == 'lrl1':
        clf = LogisticRegression(class_weight = class_weight, penalty='l1',n_jobs=-1)
    elif classifier == 'xgb':
        clf = xgb.XGBClassifier(booster = 'gbtree', nthread = -1, subsample = 1, 
                                n_estimators = 600, colsample_bytree = 1, max_depth = 6, min_child_weight = 1)
    elif classifier == 'dice':
        clf = RandomDice('1234')
    elif classifier == 'currank':
        clf = CurRank()
    elif classifier == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    precision = metrics.precision_score(test_y, pred_y, average=None) 
    recall = metrics.recall_score(test_y, pred_y, average=None)
    f1 = metrics.f1_score(test_y, pred_y, average=None)
    accuracy = metrics.accuracy_score(test_y, pred_y)
    print('precision=%s, recall=%s, f1=%s, accuracy=%.2f'%(precision,recall, f1, accuracy))
    return accuracy
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventid(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'] != eventid].to_numpy()
    test  = stagedata[stagedata['eventid'] == eventid].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


### baseline
def baseline_model():
    #1. predict with current rank, rankchg = 0
    print('[*] predict with current rank, rankchg = 0')
    pred_y_simple = np.zeros_like(test_y)
    score1 = evaluate(test_y, pred_y_simple)

    #2. predict with average rankchg (change_in_rank_all):idx = 15
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    score2 = evaluate(test_y, pred_y_avg)
    return score1, score2

def classifier_model(name='lr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_classifier(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    return score

In [4]:
#load data
suffix='-withneighbor-newfeatures-timediff'
stagedata = pd.read_csv('stage-2018%s.csv'%suffix)
stagedata.fillna(0, inplace=True)
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 35 columns):
Unnamed: 0                   805 non-null int64
target                       805 non-null int64
eventid                      805 non-null int64
car_number                   805 non-null int64
stageid                      805 non-null int64
firststage                   805 non-null int64
pit_in_caution               805 non-null int64
start_position               805 non-null int64
start_rank                   805 non-null int64
start_rank_ratio             805 non-null float64
top_pack                     805 non-null int64
bottom_pack                  805 non-null int64
average_rank                 805 non-null float64
average_rank_all             805 non-null float64
change_in_rank               805 non-null int64
change_in_rank_all           805 non-null float64
rate_of_change               805 non-null int64
rate_of_change_all           805 non-null float64
laptime_green_mean_pr

In [5]:
stagedata.head(5)

,Unnamed: 0,target,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
0,0,0,0,1,0,0,0,7,7,0.304348,...,0.000000,0,0,0.00000,0,0,0,0,0,0
1,1,-5,0,1,1,1,1,7,7,0.304348,...,3.538589,39,39,11.54325,-2,-1,1,3,0,-1
2,2,3,0,1,2,1,0,7,2,0.086957,...,7.902623,76,72,59.63585,1,0,0,-5,-5,1
3,3,-4,0,1,3,1,0,7,5,0.217391,...,6.817462,57,53,40.43850,-1,-2,-3,-6,-8,-5
4,4,0,0,1,4,1,1,7,1,0.043478,...,6.182861,56,52,39.51240,0,0,0,-1,-2,-3


In [6]:
cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(classifiers)
print('cols:%s'%cols)
retdf = pd.DataFrame([],columns=cols)

eventsname = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events = set(stagedata['eventid'])
for eventid in events:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])        
    
retdf.to_csv('crossvalid_stagedata_splitbyevent%s.csv'%suffix)
df_event = retdf

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lrl1', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Phoenix
[*] predict with currank model
precision=[0.         0.14035088 0.        ], recall=[0. 1. 0.], f1=[0.         0.24615385 0.        ], accuracy=0.14
[*] predict with avgrank model
precision=[0.48275862 0.16129032 0.16      ], recall=[0.46666667 0.3125     0.10526316], f1=[0.47457627 0.21276596 0.12698413], accuracy=0.32
[*] predict with dice model
precision=[0.52173913 0.2        0.31578947], recall=[0.4        0.375      0.31578947], f1=[0.45283019 0.26086957 0.31578947], accuracy=0.37
[*] predict with lr model
precision=[0.65079365 0.15384615 0.47368421], recall=[0.68333333 0.125      0.47368421], f1=[0.66666667 0.13793103 0.47368421], accuracy=0.54
[*] predict with lrl1 model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", 

precision=[0.64615385 0.14285714 0.48571429], recall=[0.7        0.125      0.44736842], f1=[0.672      0.13333333 0.46575342], accuracy=0.54
[*] predict with lsvc model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.64516129 0.14285714 0.44736842], recall=[0.66666667 0.125      0.44736842], f1=[0.6557377  0.13333333 0.44736842], accuracy=0.52
[*] predict with lsvcl2 model


...

precision=[1.         0.3880597  0.48275862], recall=[0.19047619 0.92857143 0.41176471], f1=[0.32       0.54736842 0.44444444], accuracy=0.46
[*] predict with lsvc model
precision=[1.         0.37142857 0.5       ], recall=[0.14285714 0.92857143 0.41176471], f1=[0.25       0.53061224 0.4516129 ], accuracy=0.44
[*] predict with lsvcl2 model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.66666667 0.5        0.34939759], recall=[0.14285714 0.21428571 0.85294118], f1=[0.23529412 0.3        0.4957265 ], accuracy=0.39
[*] predict with rf model
precision=[0.55102041 0.5        0.45714286], recall=[0.64285714 0.35714286 0.47058824], f1=[0.59340659 0.41666667 0.46376812], accuracy=0.51
[*] predict with lrbias model
[LibLinear]precision=[1.         0.39393939 0.5       ], recall=[0.19047619 0.92857143 0.44117647], f1=[0.32       0.55319149 0.46875   ], accuracy=0.47
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.58333333 0.38095238 0.46153846], recall=[0.5        0.57142857 0.35294118], f1=[0.53846154 0.45714286 0.4       ], accuracy=0.47
rec:['Gateway', 701, 104, '+:34,0:28,-:42', 0.2692307692307692, 0.3269230769230769, 0.25, 0.5, 0.46153846153846156, 0.4423076923076923, 0.3942307692307692, 0.5096153846153846, 0.47115384615384615, 0.47115384615384615]


In [7]:
df_event

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,Phoenix,691,114,"+:38,0:16,-:60",0.140351,0.324561,0.368421,0.535088,0.535088,0.517544,0.491228,0.543860,0.526316,0.412281
0,Indy500,580,225,"+:82,0:47,-:96",0.208889,0.324444,0.395556,0.560000,0.546667,0.546667,0.453333,0.435556,0.560000,0.506667
0,Texas,678,127,"+:39,0:34,-:54",0.267717,0.314961,0.291339,0.519685,0.559055,0.519685,0.464567,0.480315,0.527559,0.480315
0,Iowa,696,109,"+:42,0:28,-:39",0.256881,0.293578,0.256881,0.412844,0.403670,0.394495,0.385321,0.385321,0.403670,0.366972
0,Pocono,679,126,"+:29,0:61,-:36",0.484127,0.277778,0.238095,0.404762,0.380952,0.365079,0.222222,0.357143,0.412698,0.388889
0,Gateway,701,104,"+:34,0:28,-:42",0.269231,0.326923,0.250000,0.500000,0.461538,0.442308,0.394231,0.509615,0.471154,0.471154


In [8]:
retdf = pd.DataFrame([],columns=cols)

for stageid in range(8):
    train, test, train_x, train_y, test_x, test_y =split_by_stageid(stagedata, stageid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = ['stage%d'%stageid,train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])  
    
retdf.to_csv('crossvalid_stagedata_splitbystage%s.csv'%suffix)
df_stage = retdf

[*] predict with currank model
precision=[0.         0.25613497 0.        ], recall=[0. 1. 0.], f1=[0.         0.40781441 0.        ], accuracy=0.26
[*] predict with avgrank model
precision=[0.37172775 0.22535211 0.23115578], recall=[0.53787879 0.09580838 0.2081448 ], f1=[0.43962848 0.13445378 0.21904762], accuracy=0.31
[*] predict with dice model
precision=[0.41197183 0.25806452 0.3956044 ], recall=[0.44318182 0.28742515 0.32579186], f1=[0.4270073  0.27195467 0.3573201 ], accuracy=0.36
[*] predict with lr model
precision=[0.49633252 0.36842105 0.56603774], recall=[0.76893939 0.41916168 0.13574661], f1=[0.60326895 0.39215686 0.2189781 ], accuracy=0.46
[*] predict with lrl1 model
precision=[0.51075269 0.36619718 0.50746269], recall=[0.71969697 0.46706587 0.15384615], f1=[0.59748428 0.41052632 0.23611111], accuracy=0.46
[*] predict with lsvc model
precision=[0.48199446 0.35744681 0.25      ], recall=[0.65909091 0.50299401 0.06334842], f1=[0.5568     0.41791045 0.10108303], accuracy=0.42


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", 

precision=[0.44863014 0.30909091 0.32142857], recall=[0.49621212 0.40718563 0.20361991], f1=[0.47122302 0.35142119 0.24930748], accuracy=0.37
[*] predict with lrbias model
[LibLinear]precision=[0.49234694 0.35576923 0.48076923], recall=[0.73106061 0.44311377 0.11312217], f1=[0.58841463 0.39466667 0.18315018], accuracy=0.45
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.4375     0.30677291 0.29655172], recall=[0.42424242 0.46107784 0.19457014], f1=[0.43076923 0.36842105 0.23497268], accuracy=0.36
rec:['stage0', 153, 652, '+:221,0:167,-:264', 0.2561349693251534, 0.3128834355828221, 0.36349693251533743, 0.4647239263803681, 0.46319018404907975, 0.4171779141104294, 0.41411042944785276, 0.37423312883435583, 0.44785276073619634, 0.3558282208588957]
[*] predict with currank model
precision=[0.         0.26305609 0.        ], recall=[0. 1. 0.], f1=[0.         0.41653905 0.        ], accuracy=0.26
[*] predict with avgrank model
precision=[0.34674923 0.24242424 0.24223602], recall=[0.57435897 0.05882353 0.20967742], f1=[0.43243243 0.09467456 0.22478386], accuracy=0.31
[*] predict with dice model
precision=[0.39312977 0.22881356 0.37956204], recall=[0.52820513 0.19852941 0.27956989], f1=[0.45076586 0.21259843 0.32198142], accuracy=0.35
[*] predict with lr model
precision=[0.46511628 0.28525641 0.53781513], recall=[0.20512821 0.65441176 0.34408602], 

...

precision=[0.         0.11111111 0.28571429], recall=[0.   0.25 0.5 ], f1=[0.         0.15384615 0.36363636], accuracy=0.19
[*] predict with lsvcl2 model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


precision=[0.   0.25 0.  ], recall=[0. 1. 0.], f1=[0.  0.4 0. ], accuracy=0.25
[*] predict with rf model
precision=[0.44444444 0.2        1.        ], recall=[0.5  0.25 0.5 ], f1=[0.47058824 0.22222222 0.66666667], accuracy=0.44
[*] predict with lrbias model
[LibLinear]precision=[0.         0.1        0.16666667], recall=[0.   0.25 0.25], f1=[0.         0.14285714 0.2       ], accuracy=0.12
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', 

precision=[0.4 0.2 0.5], recall=[0.25 0.25 0.75], f1=[0.30769231 0.22222222 0.6       ], accuracy=0.38
rec:['stage7', 789, 16, '+:4,0:4,-:8', 0.25, 0.1875, 0.3125, 0.0625, 0.1875, 0.1875, 0.25, 0.4375, 0.125, 0.375]


In [9]:
#xgb max_tree_depth=3
df_stage

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,stage0,153,652,"+:221,0:167,-:264",0.256135,0.312883,0.363497,0.464724,0.463190,0.417178,0.414110,0.374233,0.447853,0.355828
0,stage1,288,517,"+:186,0:136,-:195",0.263056,0.307544,0.352031,0.373308,0.379110,0.363636,0.307544,0.427466,0.371373,0.406190
0,stage2,421,384,"+:140,0:112,-:132",0.291667,0.263021,0.317708,0.361979,0.364583,0.375000,0.351562,0.460938,0.356771,0.455729
0,stage3,547,258,"+:91,0:89,-:78",0.344961,0.228682,0.333333,0.457364,0.449612,0.453488,0.449612,0.403101,0.449612,0.426357
0,stage4,657,148,"+:48,0:53,-:47",0.358108,0.209459,0.344595,0.472973,0.479730,0.479730,0.412162,0.513514,0.466216,0.493243
0,stage5,725,80,"+:26,0:29,-:25",0.362500,0.200000,0.325000,0.512500,0.512500,0.512500,0.475000,0.500000,0.512500,0.625000
0,stage6,767,38,"+:11,0:13,-:14",0.342105,0.236842,0.263158,0.368421,0.394737,0.421053,0.315789,0.552632,0.368421,0.526316
0,stage7,789,16,"+:4,0:4,-:8",0.250000,0.187500,0.312500,0.062500,0.187500,0.187500,0.250000,0.437500,0.125000,0.375000


In [10]:
#xgb max_tree_depth=6
df_stage

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,stage0,153,652,"+:221,0:167,-:264",0.256135,0.312883,0.363497,0.464724,0.463190,0.417178,0.414110,0.374233,0.447853,0.355828
0,stage1,288,517,"+:186,0:136,-:195",0.263056,0.307544,0.352031,0.373308,0.379110,0.363636,0.307544,0.427466,0.371373,0.406190
0,stage2,421,384,"+:140,0:112,-:132",0.291667,0.263021,0.317708,0.361979,0.364583,0.375000,0.351562,0.460938,0.356771,0.455729
0,stage3,547,258,"+:91,0:89,-:78",0.344961,0.228682,0.333333,0.457364,0.449612,0.453488,0.449612,0.403101,0.449612,0.426357
0,stage4,657,148,"+:48,0:53,-:47",0.358108,0.209459,0.344595,0.472973,0.479730,0.479730,0.412162,0.513514,0.466216,0.493243
0,stage5,725,80,"+:26,0:29,-:25",0.362500,0.200000,0.325000,0.512500,0.512500,0.512500,0.475000,0.500000,0.512500,0.625000
0,stage6,767,38,"+:11,0:13,-:14",0.342105,0.236842,0.263158,0.368421,0.394737,0.421053,0.315789,0.552632,0.368421,0.526316
0,stage7,789,16,"+:4,0:4,-:8",0.250000,0.187500,0.312500,0.062500,0.187500,0.187500,0.250000,0.437500,0.125000,0.375000
